In [9]:
sys.path.append('../')
import pickle
import geopy.distance

import pandas as pd
import networkx as nx 
import matplotlib.pyplot as plt 
import numpy as np 
import trassenfinder_route_request
from helpers import StationPhillip
from time import sleep
import sqlalchemy
import requests
import re
import sys

from config import db_database, db_password, db_server, db_username

In [ ]:
engine = sqlalchemy.create_engine('postgresql://'+ db_username +':' + db_password + '@' + db_server + '/' + db_database + '?sslmode=require')

In [10]:
betriebstellen = pd.read_csv('../data/betriebsstellen.csv', index_col=False, sep=';')
betriebstellen = betriebstellen.rename(columns={'Abk':'ds100', 'Name':'name', 'Typ':'type', 'Primary location code':'location_code', 'Betr-Zust':'operation_state'})
print(len(betriebstellen))
betriebstellen = betriebstellen.drop_duplicates(subset=['ds100'], keep='first').reset_index(drop=True)
print(len(betriebstellen))
betriebstellen = betriebstellen.drop_duplicates(subset=['name'], keep='first').reset_index(drop=True)
print(len(betriebstellen))
geo_betriebstellen = pd.read_csv('../data/geo_betriebsstellen.csv', index_col=False, sep=',')
geo_betriebstellen = geo_betriebstellen.rename(columns={'KUERZEL':'ds100', 'GEOGR_BREITE':'lat', 'GEOGR_LAENGE':'lon'})
geo_betriebstellen = geo_betriebstellen.drop_duplicates(subset=['ds100'])
geo_betriebstellen = geo_betriebstellen.set_index('ds100')

23088
22984
22969


In [11]:
betriebstellen['lat'] = ''
betriebstellen['lon'] = ''
key_errors = 0
for i in range(len(betriebstellen)):
    ds100 = betriebstellen.at[i, 'ds100']
    try:
        betriebstellen.at[i, 'lat'] = geo_betriebstellen.at[ds100, 'lat']
        betriebstellen.at[i, 'lon'] = geo_betriebstellen.at[ds100, 'lon']
        continue
    except KeyError:
        key_errors += 1
        continue
    except Exception as e:
        print(e)
        continue

In [13]:
betriebstellen = betriebstellen.loc[:, ['ds100', 'name', 'type', 'lat', 'lon', 'location_code', 'operation_state']]
betriebstellen['lon'] = pd.to_numeric(betriebstellen['lon'], errors='coerce')
betriebstellen['lat'] = pd.to_numeric(betriebstellen['lat'], errors='coerce')

In [14]:
betriebstellen.to_sql('betriebstellen', if_exists='replace', method='multi', con=engine)

In [ ]:
pd.read_sql('SELECT * FROM betriebstellen', con=engine)

In [ ]:
station_df = pd.read_csv('../data/new_staions_for_streckennetz.csv', index_col=False, engine='c')
station_df.to_sql('stations', if_exists='replace', method='multi', con=engine)

In [ ]:
planned_path_df = pd.read_feather('../data_buffer/planned_path_df')

In [ ]:
for i in range(len(stations)):
    if pd.isna( stations.at[i, 'lat']) or pd.isna( stations.at[i, 'lon']):
        station_eva = stations.at[i, 'eva']
        response = requests.get('https://marudor.de/api/station/v1/station/' + str(station_eva))
        # print(response)
        response = response.json()
        try:
            # print('requesting')
            stations.at[i, 'lat'] = response['location']['latitude']
            stations.at[i, 'lon'] = response['location']['longitude']
            # print(response['location']['lat'], response['location']['lng'])
        except KeyError:
            print(stations.at[i, 'name'], response)
            # break
            continue

In [ ]:
for i in range(len(stations)):
    station_eva = stations.at[i, 'eva']

    response = requests.get('https://marudor.de/api/station/v1/iris/' + station_eva)
    # print(response)
    response = response.json()
    # print(response)
    try:
        # print('requesting')
        ds100 = response['station']['ds100']
    except KeyError:
        print(response)
        continue
    stations.at[i, 'ds100'] = ds100
    for related_station in response['relatedStations']:
        new_dict = {'eva':[related_station['eva']], 'name':[related_station['name']], 'ds100':[related_station['name']], 'lat':[''], 'lon':[''], 'mutter':[related_station['meta']]}
        new_station = pd.DataFrame(new_dict)
        if not related_station['name'] in stations['name'].values:
            stations = stations.append(new_station, ignore_index=True)
            print('found station:', related_station['name'])


In [ ]:
# station_names = stations.loc[:, 'name'].tolist()
# bar = Bar('checking if stations are in stations.csv', max=len(planned_path_df))
for number, path in enumerate(planned_path_df.iloc[:, 1]):
    if number % 1000 == 0:
        print(number)
    # bar.next()
    if path:
        path = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', ';;;', path)
        path = path.replace('{', '').replace('}', '').replace('"', '').split(',')
        for station in path:
            station = station.replace(';;;', ',')
            if not station in station_names and not station in not_found:
                # print(station)
                response = requests.get('https://marudor.de/api/hafas/v1/station/' + station.replace('/', ''))
                # print(response)
                response = response.json()
                for stations_data in response:
                    if stations_data['title'] == station:
                        new_dict = {'eva':[stations_data['id']], 'name':[stations_data['title']], 'ds100':[''], 'lat':[stations_data['coordinates']['lat']], 'lon':[stations_data['coordinates']['lng']]}
                        new_station = pd.DataFrame(new_dict)
                        stations = stations.append(new_station, ignore_index=True)
                        station_names.append(stations_data['title'])
                        break
                else:
                    not_found.append(station)
                    print('nothing found for', station)
    else:
        continue

In [ ]:
for i in range(len(stations)):
    if pd.isna( stations.at[i, 'lat']):
        station_eva = stations.at[i, 'eva']
        response = requests.get('https://marudor.de/api/station/v1/station/' + str(station_eva))
        # print(response)
        response = response.json()
        try:
            # print('requesting')
            stations.at[i, 'lat'] = response['location']['latitude']
            stations.at[i, 'lon'] = response['location']['longitude']
        except KeyError:
            print(stations.at[i, 'name'], response)
            continue

In [2]:
from pyhafas import HafasClient
from pyhafas.profile import DBProfile

client = HafasClient(DBProfile())

stations = client.locations("Tübingen")

In [3]:
stations[6]

<class 'pyhafas.types.fptf.Station'>({'id': '000752120', 'name': 'Westbahnhof, Tübingen', 'latitude': 48.520445, 'longitude': 9.039211})